In [30]:
%%capture
!pip install folium matplotlib mapclassify

In [31]:
import pandas as pd
import geopandas as gpd
import os
import zipfile

FOLDER_PATH = '/content/drive/MyDrive/DSL SMART CITIES'

from google.colab import drive
drive.mount('/content/drive')

COLS_TO_KEEP = ['fclass','geometry']

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Import NILs

In [32]:
nil = gpd.read_file(os.path.join(FOLDER_PATH, 'Processed','nil.geojson'))
nil = nil.to_crs(epsg=4326)
nil

,ID_NIL,NIL,area_sqkm,Residenti,geometry
0,1,duomo,2.341616,16608,"POLYGON ((9.19482 45.47201, 9.19473 45.47192, ..."
1,2,brera,1.637333,18294,"POLYGON ((9.19165 45.46906, 9.19204 45.46937, ..."
2,3,giardini p.ta venezia,0.249637,43,"POLYGON ((9.20090 45.47654, 9.20000 45.47696, ..."
3,4,guastalla,1.547962,15528,"POLYGON ((9.20700 45.46787, 9.20693 45.46853, ..."
4,5,porta vigentina - porta lodovica,1.135196,13548,"POLYGON ((9.20186 45.45238, 9.20078 45.45301, ..."
...,...,...,...,...,...
83,84,parco nord,1.532331,98,"POLYGON ((9.20040 45.52848, 9.20028 45.52846, ..."
84,85,parco delle abbazie,13.733841,364,"POLYGON ((9.21711 45.43187, 9.21531 45.43232, ..."
85,86,parco dei navigli,3.623149,354,"POLYGON ((9.13886 45.42855, 9.13769 45.42947, ..."
86,87,assiano,5.840942,211,"POLYGON ((9.04687 45.46276, 9.04685 45.46278, ..."


## Import  OSM_Traffic

In [33]:
osm = gpd.read_file(os.path.join(FOLDER_PATH, 'Processed', 'OSM','gis_osm_traffic_a_free_1.shp'))
osm = osm.drop_duplicates(['osm_id'])
osm = osm[COLS_TO_KEEP]
osm

,fclass,geometry
0,parking,"POLYGON ((9.12606 45.43161, 9.12629 45.43167, ..."
1,parking,"POLYGON ((9.12556 45.43871, 9.12559 45.43872, ..."
2,fuel,"POLYGON ((9.12602 45.43895, 9.12616 45.43905, ..."
3,parking,"POLYGON ((9.13596 45.43894, 9.13723 45.43933, ..."
4,parking,"POLYGON ((9.12338 45.43694, 9.12351 45.43697, ..."
...,...,...
2162,parking,"POLYGON ((9.18214 45.52830, 9.18222 45.52878, ..."
2163,parking,"POLYGON ((9.21074 45.52674, 9.21082 45.52688, ..."
2164,parking,"POLYGON ((9.20577 45.52684, 9.20579 45.52688, ..."
2165,parking,"POLYGON ((9.20406 45.52721, 9.20450 45.52797, ..."


In [34]:
osm.explore()

# Get parkings POIs

In [35]:
parkings = ['parking',
            'parking_site',
            'parking_multistorey',
            'parking_underground']

osm_parkings = osm[osm['fclass'].isin(parkings)]
osm_parkings

,fclass,geometry
0,parking,"POLYGON ((9.12606 45.43161, 9.12629 45.43167, ..."
1,parking,"POLYGON ((9.12556 45.43871, 9.12559 45.43872, ..."
3,parking,"POLYGON ((9.13596 45.43894, 9.13723 45.43933, ..."
4,parking,"POLYGON ((9.12338 45.43694, 9.12351 45.43697, ..."
5,parking,"POLYGON ((9.12250 45.43781, 9.12267 45.43785, ..."
...,...,...
2162,parking,"POLYGON ((9.18214 45.52830, 9.18222 45.52878, ..."
2163,parking,"POLYGON ((9.21074 45.52674, 9.21082 45.52688, ..."
2164,parking,"POLYGON ((9.20577 45.52684, 9.20579 45.52688, ..."
2165,parking,"POLYGON ((9.20406 45.52721, 9.20450 45.52797, ..."


In [39]:
osm_parkings = osm_parkings.to_crs(epsg=32632)
osm_parkings['parking_sqkm'] = osm_parkings.area / 10**6
osm_parkings = osm_parkings.to_crs(epsg=4326)
osm_parkings.explore()

# SJoin

In [37]:
parkings = nil.sjoin(osm_parkings, how='left').drop('fclass',axis=1)

#Groupby

In [38]:
parkings_grouped = parkings.groupby(['ID_NIL','NIL','area_sqkm','Residenti'])['parking_sqkm'].sum().reset_index()
parkings_grouped

,ID_NIL,NIL,area_sqkm,Residenti,parking_sqkm
0,1,duomo,2.341616,16608,0.016264
1,10,stazione centrale - ponte seveso,1.556019,18410,0.010221
2,11,isola,1.322887,22641,0.023688
3,12,maciachini - maggiolina,1.674918,26097,0.027049
4,13,greco - segnano,1.768603,16137,0.018417
...,...,...,...,...,...
83,85,parco delle abbazie,13.733841,364,0.028899
84,86,parco dei navigli,3.623149,354,0.001574
85,87,assiano,5.840942,211,0.017551
86,88,parco bosco in citta',7.834006,690,0.115833


In [40]:
#parkings_grouped.to_csv(os.path.join(FOLDER_PATH, 'Processed','parkings_grouped.csv'), index=False)